# AC209b / CS109b Final Project - Milestone 3
Yujiao Chen, Brian Ho, Jonathan Jay // 04/19/2017



**Traditional statistical and machine learning methods, due Wednesday, April 19, 2017**

Think about how you would address the genre prediction problem with traditional statistical or machine learning methods. This includes everything you learned about modeling in this course before the deep learning part. Implement your ideas and compare different classifiers. Report your results and discuss what challenges you faced and how you overcame them. What works and what does not? If there are parts that do not work as expected, make sure to discuss briefly what you think is the cause and how you would address this if you would have more time and resources. 

You do not necessarily need to use the movie posters for this step, but even without a background in computer vision, there are very simple features you can extract from the posters to help guide a traditional machine learning model. Think about the PCA lecture for example, or how to use clustering to extract color information. In addition to considering the movie posters it would be worthwhile to have a look at the metadata that IMDb provides. 

You could use Spark and the [ML library](https://spark.apache.org/docs/latest/ml-features.html#word2vec) to build your model features from the data. This may be especially beneficial if you use additional data, e.g., in text form.

You also need to think about how you are going to evaluate your classifier. Which metrics or scores will you report to show how good the performance is?

The notebook to submit this week should at least include:

- Detailed description and implementation of two different models
- Description of your performance metrics
- Careful performance evaluations for both models
- Visualizations of the metrics for performance evaluation
- Discussion of the differences between the models, their strengths, weaknesses, etc. 
- Discussion of the performances you achieved, and how you might be able to improve them in the future

### Overview


This week, we applied traditional statistical and machine learning methods to the problem of genre prediction within our dataset. Using a database of 990 movies, with a third each classified as horror, romance or science ficiton, we trained and tested two genre classifiers based on differently engineered and extracted features. As a whole, the work comprises:

**A model based on bag-of-word features from movie description**

In the first model (seen below) we vectorize the overview feature (i.e. the movie summary) in a bag-of-words approach in conjunction with basic metadata from the TMDB API results. We began with a gradient-boosted tree approach utilizing XGBoost, which was approximately 73% accurate. As this performance was unsatisfactory, we then took a more sophisticated approach involving PCA dimensionality reduction.

In"milestone03_yujiaochen_brianho_jonjay_part01_word" and "milestone03_yujiaochen_brianho_jonjay_part02_PCA_SVM.pdf" we perform both EDA of the word features themselves and PCA to features that account for 90% of the variance. We then employ SVM with radial basis function to classify the horror, romance and scifi movies based on this reduced feature set. The final predicting accuracy on the test set using this model is around 80%, which is a satisfactory result.

**A model based on title features**

In "milestone03_yujiaochen_brianho_jonjay_part03_titles" we build upon our analysis using word count features, by attempting to add the features of movie titles. We find that it’s possible to train a model to predict between two distant classes (romance and horror) using only simple features engineered from the title itself, such as character/word length, presence of grammatical symbols, and sentiment. We also attempt to extend the model to predict among three classes–first using title alone -- as well as integrating our established model for predicting on description word counts, although resulting improvement is not satisfactory.

**Exploratory data analysis and vizualization of movie posters**

In "milestone03_yujiaochen_brianho_jonjay_part04_color.pdf" we perform initial analysis and exploration of our poster data set on a few sample posters, with a particular emphasis on color decomposition and analysis of frequencies.

In "milestone03_yujiaochen_brianho_jonjay_part05_poster.pdf" we perform exploratory analysis and visualization of the complete poster dataset, to understand properties within the data other than the posters themselves.



In [5]:
## Import libraries
import pandas as pd
import numpy as np

import imdb
import requests
from ast import literal_eval
from xgboost import XGBClassifier
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
## Read in the data
movies = pd.read_csv("Movie subset for poster analysis_990 movies.csv")
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 22 columns):
Unnamed: 0           990 non-null int64
Unnamed: 0.1         990 non-null int64
X                    990 non-null int64
adult                990 non-null bool
backdrop_path        981 non-null object
genre_ids            990 non-null object
id                   990 non-null int64
original_language    990 non-null object
original_title       990 non-null object
overview             986 non-null object
popularity           990 non-null float64
poster_path          990 non-null object
release_date         990 non-null object
title                990 non-null object
video                990 non-null bool
vote_average         990 non-null float64
vote_count           990 non-null int64
genre_names          990 non-null object
date                 990 non-null object
year                 990 non-null int64
genres               990 non-null object
decade               990 non-null int64
dt

In [7]:
## Cleanup column names
movies.rename(columns={"Unnamed: 0":"result_id"}, inplace = True)
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 22 columns):
result_id            990 non-null int64
Unnamed: 0.1         990 non-null int64
X                    990 non-null int64
adult                990 non-null bool
backdrop_path        981 non-null object
genre_ids            990 non-null object
id                   990 non-null int64
original_language    990 non-null object
original_title       990 non-null object
overview             986 non-null object
popularity           990 non-null float64
poster_path          990 non-null object
release_date         990 non-null object
title                990 non-null object
video                990 non-null bool
vote_average         990 non-null float64
vote_count           990 non-null int64
genre_names          990 non-null object
date                 990 non-null object
year                 990 non-null int64
genres               990 non-null object
decade               990 non-null int64
dt

In [53]:
## Filter out movies with invalid information 
valid_overview = [type(i) is str for i in movies["overview"]]
movies = movies[valid_overview]
# valid_title_filter = [type(i) is str for i in movies["title"]]
# movies = movies[valid_title_filter]

movies = movies.reset_index()

movies.tail()

,index,result_id,Unnamed: 0.1,X,adult,backdrop_path,genre_ids,id,original_language,original_title,...,vote_count,genre_names,date,year,genres,decade,label,romance,horror,scifi
981,985,985,8478,15,False,/4liSXBZZdURI0c1Id1zLJo6Z3Gu.jpg,"[878, 14, 28, 12]",76757,en,Jupiter Ascending,...,2206,"[Science Fiction, Fantasy, Action, Adventure]",2015-02-04,2015,"878, 14, 28, 12",2010,scifi,0.0,0.0,1.0
982,986,986,8165,2,False,/cfVoH243KjWXV6JoLzwxqWNb23i.jpg,"[878, 12, 9648]",70981,en,Prometheus,...,4135,"[Science Fiction, Adventure, Mystery]",2012-05-30,2012,"878, 12, 9648",2010,scifi,0.0,0.0,1.0
983,987,987,7964,1,False,/jxdSxqAFrdioKgXwgTs5Qfbazjq.jpg,"[12, 28, 878]",10138,en,Iron Man 2,...,5601,"[Adventure, Action, Science Fiction]",2010-04-28,2010,"12, 28, 878",2010,scifi,0.0,0.0,1.0
984,988,988,8167,4,False,/cZkPJ0noQvcR3oCCZ4pwYZeWUYi.jpg,"[28, 53, 878]",59967,en,Looper,...,4053,"[Action, Thriller, Science Fiction]",2012-09-26,2012,"28, 53, 878",2010,scifi,0.0,0.0,1.0
985,989,989,8380,17,False,/oZY3DOlEZbEZvRxWynWkFTe4UgE.jpg,"[53, 878, 18, 9648]",157353,en,Transcendence,...,1861,"[Thriller, Science Fiction, Drama, Mystery]",2014-04-16,2014,"53, 878, 18, 9648",2010,scifi,0.0,0.0,1.0


In [54]:
## Create bag-of-words feature representation from movie summaries
corpus = movies["overview"].tolist()
vectorizer = CountVectorizer(min_df=20, stop_words="english")
words = vectorizer.fit_transform(corpus)
print words.toarray().shape
print vectorizer.get_feature_names()

## Convert to data frame
words = pd.DataFrame(words.A, columns=vectorizer.get_feature_names())

(986, 149)
[u'alien', u'american', u'away', u'based', u'battle', u'beautiful', u'begin', u'begins', u'best', u'black', u'body', u'boy', u'brother', u'business', u'car', u'century', u'child', u'children', u'city', u'come', u'comes', u'control', u'crew', u'dark', u'daughter', u'day', u'dead', u'deadly', u'death', u'destroy', u'discover', u'discovers', u'doctor', u'dr', u'earth', u'end', u'escape', u'evil', u'face', u'falls', u'family', u'father', u'fight', u'film', u'finds', u'forces', u'friend', u'friends', u'future', u'gets', u'girl', u'girlfriend', u'goes', u'good', u'government', u'group', u'help', u'high', u'home', u'horror', u'house', u'human', u'husband', u'including', u'island', u'job', u'john', u'just', u'killer', u'later', u'learns', u'left', u'life', u'like', u'lives', u'living', u'local', u'love', u'make', u'man', u'married', u'meet', u'meets', u'men', u'mission', u'mother', u'murder', u'mysterious', u'named', u'new', u'night', u'old', u'order', u'past', u'people', u'place', 

In [55]:
## Create predictors from metadata and  
X = movies[[u'adult', u'id', u'popularity',
            u'year',
            u'vote_average', u'vote_count']]

X = X.join(words, how="left", rsuffix="_word")

In [46]:
## A function to add a label for the response variable (which genre within our three categories of interest)
def classify(ids):
    if "10749" in ids:   # romance
        return "romance"
    elif "27" in ids:    # horror
        return "horror"
    elif "878" in ids:   # science fiction
        return "scifi"

## Create response from complete genre labels
movies["label"] = movies.apply(lambda x: classify(x["genre_ids"]), axis=1)
Y = movies["label"].values

In [49]:
movies.to_csv("movies_labels.csv")

In [48]:
movies.tail()

,result_id,Unnamed: 0.1,X,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,...,vote_count,genre_names,date,year,genres,decade,label,romance,horror,scifi
985,985,8478,15,False,/4liSXBZZdURI0c1Id1zLJo6Z3Gu.jpg,"[878, 14, 28, 12]",76757,en,Jupiter Ascending,In a universe where human genetic material is ...,...,2206,"[Science Fiction, Fantasy, Action, Adventure]",2015-02-04,2015,"878, 14, 28, 12",2010,scifi,0.0,0.0,1.0
986,986,8165,2,False,/cfVoH243KjWXV6JoLzwxqWNb23i.jpg,"[878, 12, 9648]",70981,en,Prometheus,A team of explorers discover a clue to the ori...,...,4135,"[Science Fiction, Adventure, Mystery]",2012-05-30,2012,"878, 12, 9648",2010,scifi,0.0,0.0,1.0
987,987,7964,1,False,/jxdSxqAFrdioKgXwgTs5Qfbazjq.jpg,"[12, 28, 878]",10138,en,Iron Man 2,With the world now aware of his dual life as t...,...,5601,"[Adventure, Action, Science Fiction]",2010-04-28,2010,"12, 28, 878",2010,scifi,0.0,0.0,1.0
988,988,8167,4,False,/cZkPJ0noQvcR3oCCZ4pwYZeWUYi.jpg,"[28, 53, 878]",59967,en,Looper,"In the futuristic action thriller Looper, time...",...,4053,"[Action, Thriller, Science Fiction]",2012-09-26,2012,"28, 53, 878",2010,scifi,0.0,0.0,1.0
989,989,8380,17,False,/oZY3DOlEZbEZvRxWynWkFTe4UgE.jpg,"[53, 878, 18, 9648]",157353,en,Transcendence,Two leading computer scientists work toward th...,...,1861,"[Thriller, Science Fiction, Drama, Mystery]",2014-04-16,2014,"53, 878, 18, 9648",2010,scifi,0.0,0.0,1.0


In [28]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
labels = ohe.fit_transform(movies["label"].reshape(-1,1))

In [50]:
new = pd.DataFrame(labels.A)
new.columns = ["romance", "horror", "scifi"]

In [45]:
movies = movies.join(new)
movies

,result_id,Unnamed: 0.1,X,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,...,vote_count,genre_names,date,year,genres,decade,label,romance,horror,scifi
0,0,3063,0,False,/haHULz59jdG4OVmqHjZ01z7HEit.jpg,"[35, 18, 10749]",164,en,Breakfast at Tiffany's,Fortune hunter Holly Golightly finds herself c...,...,769,"[Comedy, Drama, Romance]",1961-10-05,1961,"35, 18, 10749",1960,0,1.0,0.0,0.0
1,1,3463,0,False,/evAe6OMQgRkrVWxjLktYy1tIARW.jpg,"[18, 10749, 10752]",907,en,Doctor Zhivago,Doctor Zhivago is the filmed adapation of the ...,...,193,"[Drama, Romance, War]",1965-12-22,1965,"18, 10749, 10752",1960,0,1.0,0.0,0.0
2,2,2964,1,False,/vF4d4hAKMcrDE4Y6zLwUSijp32g.jpg,"[35, 18, 10749]",284,en,The Apartment,Bud Baxter is a minor clerk in a huge New York...,...,383,"[Comedy, Drama, Romance]",1960-06-15,1960,"35, 18, 10749",1960,0,1.0,0.0,0.0
3,3,3663,0,False,/sEWKMIlgUCCb2n5mT0Mf71DRKLR.jpg,"[35, 18, 10749]",37247,en,The Graduate,Recent college graduate Benjamin Braddock is s...,...,637,"[Comedy, Drama, Romance]",1967-12-21,1967,"35, 18, 10749",1960,0,1.0,0.0,0.0
4,4,3363,0,False,/soKwqxI1j5rNkVfDeuarmTHetcH.jpg,"[18, 10751, 10402, 10749]",11113,en,My Fair Lady,A misogynistic and snobbish phonetics professo...,...,269,"[Drama, Family, Music, Romance]",1964-10-21,1964,"18, 10751, 10402, 10749",1960,0,1.0,0.0,0.0
5,5,3264,1,False,/tS3ms6HM52hLLU3eRdjVPSgSS4w.jpg,"[18, 36, 10749]",8095,en,Cleopatra,Historical epic. The triumphs and tragedy of t...,...,103,"[Drama, History, Romance]",1963-06-12,1963,"18, 36, 10749",1960,0,1.0,0.0,0.0
6,6,3464,1,False,/weiACHVZa0lQ10cd3BhJm9d8YUF.jpg,"[18, 10751, 10402, 10749]",15121,en,The Sound of Music,Film adaptation of a classic Rodgers and Hamme...,...,796,"[Drama, Family, Music, Romance]",1965-03-02,1965,"18, 10751, 10402, 10749",1960,0,1.0,0.0,0.0
7,7,3863,0,False,/sc7zy4aqkA61vtDA4hHSOv4Gfu5.jpg,"[35, 18, 10749]",49225,en,Age of Consent,An elderly artist thinks he has become too sta...,...,11,"[Comedy, Drama, Romance]",1969-05-13,1969,"35, 18, 10749",1960,0,1.0,0.0,0.0
8,8,3864,1,False,/7dk68tDxfAadbsJ4aPuMOEwbC3i.jpg,"[35, 10751, 10402, 10749]",14030,en,"Hello, Dolly!","Matchmaker, Dolly Levi takes a trip to Yonkers...",...,48,"[Comedy, Family, Music, Romance]",1969-12-12,1969,"35, 10751, 10402, 10749",1960,0,1.0,0.0,0.0
9,9,3266,3,False,/pk26ubDilpxX9V7geBnxNRwAdwu.jpg,"[35, 9648, 10749, 53]",4808,en,Charade,Regina Lambert returns to Paris from a ski hol...,...,265,"[Comedy, Mystery, Romance, Thriller]",1963-12-05,1963,"35, 9648, 10749, 53",1960,0,1.0,0.0,0.0


In [60]:
X.shape, Y.shape

((986, 155), (986, 3))

In [59]:
Y = movies[["romance", "horror", "scifi"]]

In [51]:
from sklearn.grid_search import GridSearchCV

In [61]:
## Set up K-fold cross validation
kf = StratifiedKFold(Y, n_folds=5, shuffle=True)

model = XGBClassifier()

n_estimators = [100, 150, 200, 300, 400]
max_depth = [1, 2, 4, 6]
learning_rate = [0.1, 0.2, 0.3, .4, .5]
param_grid = dict(max_depth=max_depth, n_estimators=n_estimators, learning_rate = learning_rate)

grid_search = GridSearchCV(model, param_grid, n_jobs=-1, cv=kf)

grid_result = grid_search.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

IndexError: too many indices for array

In [69]:
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import confusion_matrix

rs = ShuffleSplit(len(X), n_iter=3, test_size=.25, random_state=0)

for train, test in rs:
    train_x, test_x = X.iloc[train,:], X.iloc[test,:]
    train_y, test_y = Y.iloc[train,:], Y.iloc[test,:]
    
model = XGBClassifier(n_estimators=300, max_depth=2, learning_rate=0.3)
model.fit(train_x, train_y)

y_pred = model.predict(test_x)
print confusion_matrix(test_y, y_pred)
print accuracy_score(test_y, y_pred)

ValueError: bad input shape (739, 3)

In [67]:
Y

,romance,horror,scifi
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0
5,1.0,0.0,0.0
6,1.0,0.0,0.0
7,1.0,0.0,0.0
8,1.0,0.0,0.0
9,1.0,0.0,0.0


The gradient-boosted tree model had an accuracy of 74% after 5-fold cross validation and parameter tuning. Based on the confusion matrix,it appears there were roughly equal rates of misclassification between labels. The poor rate of accuracy might be related to the relatively small size of the dataset and expansive age range — having only 990 movie across several decades could make classification more difficult. 